In [18]:
import numpy as np
import pandas as pd
import random as r
import math
import os
from datetime import timedelta
import statistics as stats 

import matplotlib.pyplot as plot
import matplotlib.ticker as mtick
import networkx as nx
import seaborn as sns

from pyomo.environ import *
from pyomo.opt import SolverFactory
#from gurobipy import GRB

ImportError: cannot import name 'glpsol' from 'pyomo.opt' (/Users/chelseagreene/opt/anaconda3/envs/epi_supplychain_env/lib/python3.8/site-packages/pyomo/opt/__init__.py)

In [2]:
#id directories
results_dir = os.getcwd()+'/outputs/strategic_decisions'

In [3]:
os.chdir(results_dir)

In [4]:
I_set = 4
K_set = 1
T_set = 12

I_K_dict = {1: [1,2,3,4]}

model = ConcreteModel()

#####define sets#######
model.K = Set(initialize = range(1,K_set+1))
model.I = Set(initialize = range(1,I_set+1))
model.T = Set(initialize = range(1,T_set+1))
model.T_0 = Set(initialize = range(0,T_set+1))
model.I_K = Set(model.K)
for k in model.K:
    for i in range(0,len(I_K_dict[k])):
        model.I_K[k].add(I_K_dict[k][i])

In [5]:
budget_levels = range(0,30+1)
level_amount_incrememented = 10000

reliability_levels = [.5,.75,.95]

warehouse_cap_level = 1 #time intervals average amount of supply that can be stored

trade_off_dict = {}

In [6]:
def triangular_dist_demand(a,b,c,prob):
    if prob <= (c-a)/(b-a):
        return(a+((((prob)*(b-a)*(c-a)))**(1/2)))
    else:
        return(b-(((1-prob)*(b-a)*(b-c))**(1/2)))

In [7]:
def triangular_dist_supply(a,b,c,prob):
    if 1-prob <= (c-a)/(b-a):
        return(a+((((1-prob)*(b-a)*(c-a)))**(1/2)))
    else:
        return(b-(((prob)*(b-a)*(b-c))**(1/2)))

In [8]:
def lead_time_dist(a,b,c):
    lead_time_dist_array = np.zeros(T_set+1)
    cum_prob = 0
    t = 0
    
    while t < b+1:
        if t+1 <= a:
            t = t+1 
        elif (a < t+1 & t+1 <= c):
            cum_prob_temp = ((t+1-a)**2)/((b+1-a)*(c-a))
            prob = cum_prob_temp - cum_prob
            lead_time_dist_array[t] = prob
            cum_prob = cum_prob_temp
            t = t+1
        elif c < t+1 & t+1 <= b+1:
            cum_prob_temp = 1-((b+1-(t+1))**2)/((b+1-a)*(b+1-c))
            prob = cum_prob_temp - cum_prob
            lead_time_dist_array[t] = prob
            cum_prob = cum_prob_temp
            t = t+1
    
    return(lead_time_dist_array)

In [9]:
#define demand attributes      
dem_dictionary_k = {1: [40000, 80000, 50000]
}

#define supply attributes
#capacity
cap_dictionary_supplier_SKU = {1: [9000,11000,10000], #min, max, likely, a,b,c
                     2: [10000,20000,15000], 
                     3: [17000,20000,17000],
                     4: [40000,100000,60000]}


#lead-time
lead_time_dictionary_supplier_SKU = {1: [3,5,4], #min, max, likely, a,b,c
                     2: [1,3,2], 
                     3: [4,6,3],
                     4: [0,2,0]}

#dynamic adj
dynamic_adj_dem = [0.5, 0.55, 0.6, 1, 1.1, 1.5, 1.6, 1.2, 1.1, 0.85]
dynamic_adj_supply = np.zeros(T_set)
for t in range(1, T_set+1):
    if dynamic_adj_dem[t-1] < 1:
        dynamic_adj_supply[t-1] = 1+(1-dynamic_adj_dem[t-1])
    else:
        dynamic_adj_supply[t-1] = 2-dynamic_adj_dem[t-1]

budget = 1700000
reliability_level = .75
dynamic = True

#supplier-SKU names
names_supplier_SKU = ['Supplier 1 - SKU: N95-MG', 'Supplier 1 - SKU: N95-N', 
 'Supplier 2 - SKU: SUR-MG', 'Supplier 2 - SKU: SUR-N']

In [10]:
def initialize_model(reliability_level, 
                     budget_level):

    #define parameters
    k = 1

    ###health care commodity metric parameters####
    #rank
    r_k_dict = {
        1 : 1
    }

    #quality 
    q_i_dict = {
        1 : 1,
        2: .5,
        3: .5,
        4: .2
    }


    ####Supply Parameters#####

    #lead time (supplier 3 has the lowest lead time)
    f_i_tDiff_dict = {}
    for i in range(1,I_set+1):
        lead_time_dist_array = lead_time_dist(lead_time_dictionary_supplier_SKU.get(i)[0],
                                             lead_time_dictionary_supplier_SKU.get(i)[1],
                                             lead_time_dictionary_supplier_SKU.get(i)[2])
        for t in range(0,T_set+1):
                    f_i_tDiff_dict[tuple([i,t])] = lead_time_dist_array[t]


    #incoming orders
    o_i_t_init_dict = {}


    for i in range(1,I_set+1):
        for t in range(1,T_set+1):
            if t <= 2:
                o_i_t_init_dict[tuple([i,t])] = 0 #20000/4 #5000
            else:
                o_i_t_init_dict[tuple([i,t])] = 0



    #lead time (supplier 1 has the higest capacity)
    CAP_i_t_dict = {}

    for i in range(1,I_set+1):
        cap_temp = triangular_dist_supply(cap_dictionary_supplier_SKU.get(i)[0],
                                          cap_dictionary_supplier_SKU.get(i)[1],
                                          cap_dictionary_supplier_SKU.get(i)[2],
                                          reliability_level)
        if (dynamic == True):
            for t in range(1,T_set+1):
                cap_temp_2 = cap_temp*dynamic_adj_supply[t-1]
                CAP_i_t_dict[tuple([i,t])] = cap_temp_2
        else:
            for t in range(1,T_set+1):
                CAP_i_t_dict[tuple([i,t])] = cap_temp

    ####Demand Parameters####
    n_k_init_dict = {
        1 : 0
    }


    #demand RV
    D_k_t_dict = {}
    demand_temp = triangular_dist_demand(dem_dictionary_k.get(k)[0],
                                         dem_dictionary_k.get(k)[1],
                                         dem_dictionary_k.get(k)[2],
                                         reliability_level)

    if (dynamic == True):
        for t in range(1,T_set+1):
            demand_temp_2 =demand_temp*dynamic_adj_dem[t-1]
            D_k_t_dict[tuple([k,t])] = demand_temp_2
    else:
        for t in range(1,T_set+1):
            D_k_t_dict[tuple([k,t])] = demand_temp


    #starting inventory
    z_i_init_dict = {
        1 : 8000,
        2 : 8000,
        3 : 8000,
        4 : 8000
    }

    #size
    s_i_dict = {
        1 : 1,
        2 : 1,
        3 : 1,
        4 : 1
    }

    #warehouse capacity
    h = 80000*warehouse_cap_level 

    #cost (supplier 2 has the lowest cost)
    c_i_dict = {
        1 : 10,
        2 : 5,
        3 : 6,
        4 : .8
    }

    b = 100000*budget_level
    
    model = ConcreteModel()

    #####define sets#######
    model.K = Set(initialize = range(1,K_set+1))
    model.I = Set(initialize = range(1,I_set+1))
    model.T = Set(initialize = range(1,T_set+1))
    model.T_0 = Set(initialize = range(0,T_set+1))

    ####initialize parameters####

    def rank_param_initialize(model, k):
        return(r_k_dict.get(k))

    model.r_k = Param(model.K, initialize = rank_param_initialize)

    def discount_param_initialize(model, k, t):
        discount_temp = 1/model.r_k[k]
        numerator = ((1+discount_temp)**T_set)
        denominator = ((1+discount_temp)**(t-1))

        return(numerator/denominator)

    model.tau_k_t = Param(model.K, model.T, initialize = discount_param_initialize)

    #model.tau_k_t.pprint()

    def quality_param_initialize(model, i):
        return(q_i_dict.get(i))

    model.q_i = Param(model.I, initialize = quality_param_initialize)

    #model.q_k_i.pprint()

    def delay_dist_param_initialize(model, i, t):
        return(f_i_tDiff_dict.get(tuple([i,t])))

    model.f_i_tDiff = Param(model.I, model.T_0, 
                              initialize = delay_dist_param_initialize)

    #model.f_k_i_tDiff.pprint()

    def incoming_orders_initialize(model, i, t):
        return(o_i_t_init_dict.get(tuple([i,t])))

    model.o_i_t_init = Param(model.I, model.T, initialize = incoming_orders_initialize)

    #model.o_k_i_t.pprint()

    def supplier_cap_param_initialize(model, i, t):
        return(CAP_i_t_dict.get(tuple([i,t])))

    model.cap_i_t = Param(model.I, model.T, initialize = supplier_cap_param_initialize)

    #model.cap_k_i_t.pprint()

    def unfulfilled_demand_param_initialize(model, k):
        return(n_k_init_dict.get(k))

    model.n_k_init = Param(model.K, initialize = unfulfilled_demand_param_initialize)

    #model.n_k_intialize.pprint()

    #assume preparing for one sd above the mean
    def demand_param_initialize(model, k, t):
        return(D_k_t_dict.get(tuple([k,t])))

    model.d_k_t = Param(model.K, model.T, initialize = demand_param_initialize)

    #model.d_k_t.pprint()

    def beg_inv_param_initialize(model, i):
        return(z_i_init_dict.get(i))

    model.z_i_init = Param(model.I, initialize = beg_inv_param_initialize)

    #model.z_k_i_init.pprint()

    def item_size_param_initialize(model, i):
        return(s_i_dict.get(i))

    model.s_i = Param(model.I, initialize = item_size_param_initialize)

    #model.s_k.pprint()

    def warehouse_cap_param_initialize(model):
        return(h)

    model.h = Param(initialize = h)

    #model.h.pprint()

    def cost_param_initialize(model, i):
        return(c_i_dict.get(1))

    model.c_i = Param(model.I, initialize = cost_param_initialize)

    #model.c_k_i.pprint()

    def budget_param_initialize(model):
        return(b)

    model.b = Param(initialize = budget_param_initialize)

    #model.b.pprint()

    ####initialize variables####
    model.x_i_t = Var(model.I, model.T, within = NonNegativeReals) #amount ordered
    model.y_i_t = Var(model.I, model.T, within = NonNegativeReals) #amount recieved
    model.z_i_t = Var(model.I, model.T, within = NonNegativeReals)
    model.n_k_t = Var(model.K, model.T, within = NonNegativeReals)
    model.m_k_i_t = Var(model.K, model.I, model.T, within = NonNegativeReals)
    
    return(model)

In [11]:
def initialize_objective(model):
    model.Objective = Objective(expr = 
                                (sum(model.q_i[i]*sum(model.tau_k_t[k,t]*model.m_k_i_t[k,i,t] 
                                                          for t in model.T) 
                                     for i in model.I for k in model.K)),
                                sense = maximize)
    
    return(model)

In [12]:
def initialize_constraints(model):
    def initalize_unsatisfied_demand_constraint_initialize(model, k):
        return(model.n_k_t[k,1] == model.n_k_init[k])

    model.initalize_unsatisfied_demand_constraint= \
    Constraint(model.K, rule = initalize_unsatisfied_demand_constraint_initialize)

    def calculate_unsatisfied_demand_constraint_initialize(model,k,t):
        if (t == 1):
            return(sum(model.m_k_i_t[k,i,t] for i in model.I) <= model.n_k_t[k,t] + model.d_k_t[k,t])
        else:
            return(model.n_k_t[k,t] == model.n_k_t[k,t-1] + model.d_k_t[k,t] \
                   - sum(model.m_k_i_t[k,i,t] for i in model.I))

    model.calculate_unsatisfied_demand_constraint = \
    Constraint(model.K, model.T, rule = calculate_unsatisfied_demand_constraint_initialize)

    def supplier_constraint_initialize(model, i, t):
        return(model.x_i_t[i,t] <= model.cap_i_t[i,t])

    model.supplier_constraint = \
    Constraint( model.I, model.T, rule = supplier_constraint_initialize)

    def incoming_orders_constraint_initialize(model, i, t):
        return(model.y_i_t[i,t] - 
               sum(model.f_i_tDiff[i, t-t_ordered_time]*model.x_i_t[i,t_ordered_time] 
                   for t_ordered_time in range(1,t+1)) - model.o_i_t_init[i,t] == 0)

    model.incoming_ordered_constraint = Constraint(model.I, model.T, 
                                                   rule = incoming_orders_constraint_initialize)

    def beggining_inventory_constraint_initialize(model, i):
        return(model.z_i_t[i,1] == model.z_i_init[i])

    model.beggining_inventory_constraint = Constraint(model.I, 
                                                      rule = beggining_inventory_constraint_initialize)

    def calculate_available_inventory_constraint_initialize(model, i, t):
        if (t > 1):
            return(model.z_i_t[i,t] == model.z_i_t[i,t-1] + model.y_i_t[i,t-1] - \
                   sum(model.m_k_i_t[k,i,t-1] for k in model.K))
        else:
            return(Constraint.Skip)

    model.calculate_available_inventory_constraint = Constraint(model.I, model.T,
                                                                rule = 
                                                                calculate_available_inventory_constraint_initialize)

    def cannot_fulfill_more_than_available_constraint_initialize(model, i, t):
        return(sum(model.m_k_i_t[k,i,t] for k in model.K) - model.z_i_t[i,t] <= 0)

    model.cannot_fulfill_more_than_available_constraint = \
    Constraint(model.I, model.T, rule = cannot_fulfill_more_than_available_constraint_initialize)

    def warehouse_capacity_constraint_initialize(model, t):
        return(sum(model.s_i[i]*model.z_i_t[i,t] for i in model.I) <= model.h)

    model.warehouse_capacity_constraint = \
    Constraint(model.T, rule = warehouse_capacity_constraint_initialize)

    def budget_constraint_initialize(model):
        return(sum(model.c_i[i]*model.x_i_t[i,t] 
                   for i in model.I for t in model.T) 
               <= model.b)

    model.budget_constraint = \
    Constraint(rule = budget_constraint_initialize)
    
    return(model)

In [13]:
def extract_data(model, reliability_level, budget, 
                 unsatisfied_df, fulfilled_df, ordered_df, actual,
                HCC_dict):
    
    #extract from unsatisfied_df
    def unsatisfied_df_update(unsatisfied_df):
        if unsatisfied_df.empty:
            unsatisfied_df = pd.DataFrame(list(model.n_k_t.extract_values().items()),\
                                          columns = ['sets','units_unsatisfied'])
            unsatisfied_df = unsatisfied_df[unsatisfied_df['units_unsatisfied'].notnull()]            
            unsatisfied_df.loc[:,'item_type']=unsatisfied_df.sets.map(lambda x:x[0])
            unsatisfied_df.loc[:,'time_interval']=unsatisfied_df.sets.map(lambda x:x[1])
            unsatisfied_df = unsatisfied_df[['item_type', 'time_interval', 
                                             'units_unsatisfied']]
            unsatisfied_df ['reliability_level'] = [reliability_level]*len(unsatisfied_df)
            unsatisfied_df['budget'] = [budget]*len(unsatisfied_df)
            return(unsatisfied_df)
        else:
            unsatisfied_df_temp = pd.DataFrame(list(model.n_k_t.extract_values().items()),\
                                          columns = ['sets','units_unsatisfied'])
            unsatisfied_df_temp.loc[:,'item_type']=unsatisfied_df_temp.sets.map(lambda x:x[0])
            unsatisfied_df_temp.loc[:,'time_interval']=unsatisfied_df_temp.sets.map(lambda x:x[1])
            unsatisfied_df_temp = unsatisfied_df_temp[['item_type', 'time_interval', 
                                                       'units_unsatisfied']]
            unsatisfied_df_temp ['reliability_level'] = [reliability_level]*len(unsatisfied_df_temp)
            unsatisfied_df_temp['budget'] = [budget]*len(unsatisfied_df_temp)
            unsatisfied_df_temp = unsatisfied_df_temp[unsatisfied_df_temp['units_unsatisfied'].notnull()]

            return(unsatisfied_df.append(unsatisfied_df_temp, ignore_index=True))
        
    #extract from satisfied_df
    def fulfilled_df_update(fulfilled_df):
        if fulfilled_df.empty:
            fulfilled_df = pd.DataFrame(list(model.m_k_i_t.extract_values().items()),columns = ['sets','units_fulfilled'])
            fulfilled_df.loc[:,'item_type']=fulfilled_df.sets.map(lambda x:x[0])
            fulfilled_df.loc[:,'supplier']=fulfilled_df.sets.map(lambda x:x[1])
            fulfilled_df.loc[:,'time_interval']=fulfilled_df.sets.map(lambda x:x[2])
            fulfilled_df['reliability_level'] = [reliability_level]*len(fulfilled_df)
            fulfilled_df['budget'] = [budget]*len(fulfilled_df)
            fulfilled_df = fulfilled_df[fulfilled_df['units_fulfilled'].notnull()]
            return(fulfilled_df)
        else:
            fulfilled_df_temp = pd.DataFrame(list(model.m_k_i_t.extract_values().items()),columns = ['sets','units_fulfilled'])
            fulfilled_df_temp.loc[:,'item_type']=fulfilled_df_temp.sets.map(lambda x:x[0])
            fulfilled_df_temp.loc[:,'supplier']=fulfilled_df_temp.sets.map(lambda x:x[1])
            fulfilled_df_temp.loc[:,'time_interval']=fulfilled_df_temp.sets.map(lambda x:x[2])
            fulfilled_df_temp['reliability_level'] = [reliability_level]*len(fulfilled_df_temp)
            fulfilled_df_temp['budget'] = [budget]*len(fulfilled_df_temp)
            fulfilled_df_temp = fulfilled_df_temp[fulfilled_df_temp['units_fulfilled'].notnull()]

            return(fulfilled_df.append(fulfilled_df_temp, ignore_index=True))
        
    def ordered_df_update(ordered_df):
        if ordered_df.empty:
            ordered_df = pd.DataFrame(list(model.x_i_t.extract_values().items()),columns = ['sets','units_to_order'])
            ordered_df.loc[:,'supplier']=ordered_df.sets.map(lambda x:x[0])
            ordered_df.loc[:,'time_interval']=ordered_df.sets.map(lambda x:x[1])
            ordered_df['reliability_level'] = [reliability_level]*len(ordered_df)
            ordered_df['budget'] = [budget]*len(ordered_df)
            
            return(ordered_df)
        else:
            ordered_df_temp = pd.DataFrame(list(model.x_i_t.extract_values().items()),columns = ['sets','units_to_order'])
            ordered_df_temp.loc[:,'supplier']=ordered_df_temp.sets.map(lambda x:x[0])
            ordered_df_temp.loc[:,'time_interval']=ordered_df_temp.sets.map(lambda x:x[1])
            ordered_df_temp['reliability_level'] = [reliability_level]*len(ordered_df_temp)
            ordered_df_temp['budget'] = [budget]*len(ordered_df_temp)
            
            return(ordered_df.append(ordered_df_temp, ignore_index = True))
    
    HCC_dict[tuple([budget, reliability_level])] = model.Objective.expr()
    
    unsatisfied_df = unsatisfied_df_update(unsatisfied_df)
    fulfilled_df = fulfilled_df_update(fulfilled_df)
    ordered_df = ordered_df_update(ordered_df)
    
    return(unsatisfied_df, fulfilled_df, ordered_df, HCC_dict)

In [16]:
fulfilled_df_plan = pd.DataFrame()
unsatisfied_df_plan = pd.DataFrame()
ordered_df_plan = pd.DataFrame()
HCC_dict = {}

for r in reliability_levels:
    
    objective_itr = [0,1]
    b = 1
    while(objective_itr[b-1] < objective_itr[b]):    
        model = initialize_model(r, b)
        model = initialize_objective(model)
        model = initialize_constraints(model)
        #opt = SolverFactory('gurobi_persistent')
        #opt.set_instance(model)
        #opt.solve(model)
        opt = pyomo.opt.SolverFactory("glpk")
        results = opt.solve(model)
        opt.solve(model)
        
        if (objective_itr[b-1] < objective_itr[b]):
            #extract data
            trade_off_dict[tuple([r,b*100000])] = value(model.Objective) #model.Objective.value()
            objective_itr.append(value(model.Objective))#model.Objective.value())

            unsatisfied_df_plan, fulfilled_df_plan, ordered_df_plan, HCC_dict = \
            extract_data(model, r, b*100000, unsatisfied_df_plan, 
                         fulfilled_df_plan, ordered_df_plan, 'NA',
                         HCC_dict)
        
        b = b + 1

    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

In [ ]:
#with open('test.csv', 'w') as f:
#    for key in trade_off_dict.keys():
#        f.write("%s,%s\n"%(key,trade_off_dict[key]))

In [2]:
budget = []
reliability = []
obj_value = []

for k in trade_off_dict.keys():
    obj_value.append(trade_off_dict.get(k))
    budget.append(k[1])
    reliability.append("{:.0%}".format(k[0]))

NameError: name 'trade_off_dict' is not defined

In [ ]:
plot.rcParams['legend.title_fontsize'] = 'Large'

In [ ]:
trade_off_df = pd.DataFrame([budget,reliability, obj_value]).transpose()
trade_off_df.columns = ['budget', 'reliability', 'obj_value']

In [ ]:
fulfilled_df_item_time = fulfilled_df_plan.groupby(['item_type', 'time_interval', 
                      'reliability_level', 'budget'])['units_fulfilled'].sum().reset_index()

unsatisfied_df_item_time = unsatisfied_df_plan.groupby(['item_type', 'time_interval', 
                      'reliability_level', 'budget'])['units_unsatisfied'].sum().reset_index()

reliability_df = pd.merge(fulfilled_df_item_time[['item_type', 'time_interval', 'units_fulfilled',
                                                  'reliability_level', 'budget']], 
                          unsatisfied_df_item_time[['item_type', 'time_interval', 'units_unsatisfied',
                                          'reliability_level', 'budget']], 
                          how='left', on=['item_type','time_interval',
                                         'reliability_level', 'budget'])

reliability_df['total_number_of_open_requests'] = reliability_df['units_fulfilled'] +\
reliability_df['units_unsatisfied'] 

reliability_df['percent_unsatisfied'] = \
reliability_df['units_unsatisfied']/reliability_df['total_number_of_open_requests']

In [ ]:
unsatisfied_df_plan[['item_type', 'time_interval', 'units_unsatisfied',
                                          'reliability_level', 'budget']]

In [ ]:
reliability_df_grouped = \
reliability_df.groupby(['reliability_level', 'budget'])['percent_unsatisfied'].sum().reset_index()

In [ ]:
reliability_df = pd.merge(fulfilled_df_item_time[['item_type', 'time_interval', 'units_fulfilled',
                                                  'reliability_level', 'budget']], 
                          unsatisfied_df_plan[['item_type', 'time_interval', 'units_unsatisfied',
                                          'reliability_level', 'budget']], 
                          how='left', on=['item_type','time_interval',
                                         'reliability_level', 'budget'])

In [ ]:
reliability_df_grouped['risk_of_understock'] = (reliability_df_grouped['percent_unsatisfied']/(T_set*K_set))

In [ ]:
reliability_df_grouped['budget_thousands'] = reliability_df_grouped['budget']/1000
reliability_df_grouped = reliability_df_grouped[reliability_df_grouped['budget_thousands'] <= 1300]
reliability_df_grouped['percent_unsatisfied'] = (reliability_df_grouped['risk_of_understock']*100).astype(int)

In [ ]:
os.getcwd()

In [ ]:
fig, ax = plot.subplots(figsize=(6,6))


ax.plot('budget_thousands', 'percent_unsatisfied', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == reliability_levels[0]],
         marker='X', color = 'black', label = '50%', markersize=8)#r'$\theta^D = \theta^{CAP}$ = 0' + ' (Assuming Expected Demand and Supplier Capacity)')
ax.plot('budget_thousands', 'percent_unsatisfied', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == reliability_levels[1]],
         marker='o', color = 'black', label = '75%', markersize=8, mfc = 'white') #r'$\theta^D = \theta^{CAP}$ = 1' + ' (Assuming Some Variability in Demand and Supplier Capacity)')
ax.plot('budget_thousands', 'percent_unsatisfied', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == reliability_levels[2]],
         marker='D', color = 'gray', label = '95%', markersize = 8)#r'$\theta^D = \theta^{CAP}$ = 2' + ' (high reliability)')

ax.set_xlabel('Budget (in thousands)', fontsize = 13)
ax.set_ylabel('Overall Percent of Unsatisfied Demand', fontsize = 13)

#ax.set(ylim=(0, 1), xlim=(0, max(reliability_df_grouped['budget'])))
ax.set(ylim=(0, 100), xlim=(0, 1500))
#ax.set_title(r'$\bf{Overall ~ Expected ~Percent ~of ~Unsatisfied ~Demand}$'+
#             '\n'+ r'$\bf{Under ~Various~ Budgets ~and ~Variability~ in~Demand ~and ~Supplier~Capacity~Assumptions~}$' +
#             '\n \n \n where the expected percent of unsatisfied demand for each item during week'#+ r'$(t)$' + ', represented by: \n'+\
              #r'$\psi_{k,t} = \frac{\alpha_{k,t}}{\alpha_{k,t} + \sum_{i \in I} \beta_{k,i,t}} \forall k \in K, t \in T$' +
#             '\n ...is averaged over all items and time intervals to obtain the '#+  r'$(K)$'+ ' and time intervals '+  r'$(T)$'+ ' to obtain the:'\
#             '\n overall expected percent of unsatisfied demand',# + r'$\frac{\sum_{t \in T} \sum_{k \in K} \psi_{k,t}}{K \times T}$',
#             fontsize = 16)

ax.yaxis.set_major_formatter(mtick.PercentFormatter())

plot.legend(title = 'Demand and Supply \n Capacity Reliability', 
            fontsize=13, 
            fancybox=True)._legend_box.align='center'

plot.grid(color='gray', linestyle='-', linewidth=.4, axis = 'y')


#os.chdir(os.getcwd()+'/outputs/3_2_1_strategic_decisions')
#plot.savefig('budget_high_WH.jpg')
plot.savefig('budget_low_WH.jpg')

In [ ]:
stats_df = fulfilled_df_plan[(fulfilled_df_plan['reliability_level'] == .75)&
                 (fulfilled_df_plan['budget'] >= 900000)&
                 (fulfilled_df_plan['budget'] <= 1300000)&
                            ((fulfilled_df_plan['units_fulfilled'] > 0))].reset_index()
stats_df = stats_df[['budget','supplier', 'time_interval', 'units_fulfilled']]
stats_df['quality'] = [model.q_i[x] for x in stats_df['supplier']]
stats_df['quality_norm'] = stats_df['quality']*stats_df['units_fulfilled']

In [ ]:
model.q_i.pprint()

In [ ]:
stats_df_grouped = stats_df.groupby(['budget'])\
.agg({'quality_norm':'sum', 
     'units_fulfilled':'sum'}).reset_index()

In [ ]:
stats_df_grouped

In [ ]:
overall_percent_stat_df = reliability_df_grouped[(reliability_df_grouped['reliability_level']==.75) 
                       & (reliability_df_grouped['budget']>=900000)]

In [ ]:
stats_df_grouped = stats_df_grouped.merge(overall_percent_stat_df[['budget', 'risk_of_understock']])

In [ ]:
stats_df_grouped

In [ ]:
stats_df_grouped['quality_norm'] = stats_df_grouped['quality_norm']/stats_df_grouped['units_fulfilled']

In [ ]:
stats_df_grouped

In [ ]:
len(stats_df_grouped)

In [ ]:
def triangular_dist_demand(a,b,c,prob):
    if prob <= (c-a)/(b-a):
        return(a+((((prob)*(b-a)*(c-a)))**(1/2)))
    else:
        return(b-(((1-prob)*(b-a)*(b-c))**(1/2)))

In [ ]:
dem_dictionary_k = {1: [40000, 80000, 50000]}

#demand RV
D_k_t = np.zeros(T_set)

demand_temp = triangular_dist_demand(dem_dictionary_k.get(1)[0],
                                     dem_dictionary_k.get(1)[1],
                                     dem_dictionary_k.get(1)[2],
                                     .75)

for t in range(T_set):
    demand_temp_2 =demand_temp*dynamic_adj_dem[t]
    if t == 0:
        D_k_t[t] = demand_temp_2
    else:
        D_k_t[t] = demand_temp_2 + D_k_t[t-1]

In [ ]:
D_k_t2 = []


for b in range(len(stats_df_grouped)):
    for d in D_k_t:
        D_k_t2.append(d)

In [ ]:
D_k_t2

In [ ]:
delay_stats_df = stats_df[stats_df['budget']>=900000][['budget', 
                                       'time_interval', 
                                       'units_fulfilled']].groupby(['budget', 
                                                                    'time_interval']).sum('units_fulfilled').reset_index()

In [ ]:
stats_df_grouped

In [ ]:
cum_fulfilled = []

for b in stats_df_grouped['budget']:
    temp = delay_stats_df[delay_stats_df['budget'] == b]
    fulfilled_counter = 0
    for t in range(T_set):
        temp2 = temp[temp['time_interval'] == t+1]
        
        fulfilled_counter = fulfilled_counter + temp2['units_fulfilled'].values
        
        cum_fulfilled.append(fulfilled_counter)

In [ ]:
len(D_k_t2)

In [ ]:
delay_stats_df

In [ ]:
delay_stats_df['cum_demand'] = D_k_t2
delay_stats_df['cum_fulfilled'] = pd.DataFrame(cum_fulfilled)

In [ ]:
delay_stats_df['n_delayed'] = delay_stats_df['cum_demand'] - delay_stats_df['cum_fulfilled']

In [ ]:
stats_df_grouped

In [ ]:
delay_stats_df.loc[delay_stats_df['time_interval'] ==T_set , 
                   'n_delay0'] = 0

delay_stats_df.loc[delay_stats_df['time_interval'] < T_set , 
                   'n_delay0'] = delay_stats_df['n_delayed']

In [ ]:
delay_calc = []

for b in stats_df_grouped['budget']:
    temp = delay_stats_df[delay_stats_df['budget'] == b]
    temp2 = max(temp['cum_fulfilled'])
    
    delay_calc.append(sum(temp['n_delay0'])/temp2)

In [ ]:
delay_calc

In [ ]:
perc_improv = []
for b in stats_df_grouped['budget']:
    start = HCC_dict.get(tuple([b-100000, .75]))/1000
    finish = HCC_dict.get(tuple([b, .75]))/1000
    
    print(((finish-start)/start))

In [ ]:
stats_df_grouped['budget']

In [ ]:
delay_stats_df